In [59]:
# import ssi_fc_trading
import os, sys
import pandas as pd
from ssi_fc_data import fc_md_client, model
sys.path.append('../fc-data.py/')
import configDataSSL

client = fc_md_client.MarketDataClient(configDataSSL)
data = client.intraday_ohlc(configDataSSL, model.intraday_ohlc('VN30F1M', '01/12/2023', '19/12/2023', 1, 9999, True, 1))
data = data['data']
data = pd.DataFrame(data)
data['Time'] = data['TradingDate'] + " " + data['Time']
# data.set_index('Time', inplace=True)
# Chuyển định dạng số
data['Open'] = pd.to_numeric(data['Open'], errors='coerce')
data['High'] = pd.to_numeric(data['High'], errors='coerce')
data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['Volume'] = pd.to_numeric(data['Volume'], errors='coerce')



In [60]:
data

,Symbol,Value,TradingDate,Time,Open,High,Low,Close,Volume
0,VN30F1M,1080,01/12/2023,01/12/2023 09:00:58,1080.9,1081.2,1080.0,1080.0,3446
1,VN30F1M,1080.70,01/12/2023,01/12/2023 09:01:58,1080.0,1081.0,1079.3,1080.7,1746
2,VN30F1M,1081.10,01/12/2023,01/12/2023 09:02:58,1080.8,1081.3,1080.8,1081.1,1189
3,VN30F1M,1081.40,01/12/2023,01/12/2023 09:03:58,1080.8,1081.4,1080.8,1081.4,1122
4,VN30F1M,1081.80,01/12/2023,01/12/2023 09:04:58,1081.2,1082.0,1081.2,1081.8,1237
...,...,...,...,...,...,...,...,...,...
2966,VN30F1M,1085.60,19/12/2023,19/12/2023 09:50:56,1085.7,1085.7,1085.5,1085.6,756
2967,VN30F1M,1085.40,19/12/2023,19/12/2023 09:51:58,1085.6,1085.7,1085.4,1085.4,1019
2968,VN30F1M,1085.10,19/12/2023,19/12/2023 09:52:58,1085.4,1085.5,1085.1,1085.1,946
2969,VN30F1M,1084.80,19/12/2023,19/12/2023 09:53:58,1085.2,1085.2,1084.8,1084.8,866


In [61]:

# Chuyển đổi dữ liệu thành DataFrame
custom_timeframe = 5
# Chia nhỏ dữ liệu thành các khung thời gian 45 phút
data.index = pd.to_numeric(data.index, errors='coerce')

print(data.count())  # Đây sẽ in ra số lượng cho mỗi cột
sodu = data.count()[0] % 5  # Lấy số lượng từ cột đầu tiên và tính số dư
print(data.iloc[sodu-1:-1])  # Sử dụng iloc để truy cập dữ liệu và đến bản ghi cuối cùng
data = data.iloc[sodu-1:-1]

data['custom_timeframe'] = data.index // (custom_timeframe / 1)  # Tính toán số cụm 45 phút

df_grouped = data.groupby('custom_timeframe', as_index=False).agg({
    'Time': 'last',
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
})

# Chuyển đổi thời gian thành định dạng datetime
# df_grouped['Time'] = pd.to_datetime(df_grouped['Time'])
df_grouped['Time'] = pd.to_datetime(df_grouped['Time'], format='%d/%m/%Y %H:%M:%S')

# In ra dữ liệu OHLC
# print(df_grouped)
# print(df_grouped[['Time', 'Open', 'High', 'Low', 'Close', 'Volume']])

Symbol         2971
Value          2971
TradingDate    2971
Time           2971
Open           2971
High           2971
Low            2971
Close          2971
Volume         2971
dtype: int64
       Symbol    Value TradingDate                 Time    Open    High  \
0     VN30F1M     1080  01/12/2023  01/12/2023 09:00:58  1080.9  1081.2   
1     VN30F1M  1080.70  01/12/2023  01/12/2023 09:01:58  1080.0  1081.0   
2     VN30F1M  1081.10  01/12/2023  01/12/2023 09:02:58  1080.8  1081.3   
3     VN30F1M  1081.40  01/12/2023  01/12/2023 09:03:58  1080.8  1081.4   
4     VN30F1M  1081.80  01/12/2023  01/12/2023 09:04:58  1081.2  1082.0   
...       ...      ...         ...                  ...     ...     ...   
2965  VN30F1M  1085.70  19/12/2023  19/12/2023 09:49:58  1085.6  1085.9   
2966  VN30F1M  1085.60  19/12/2023  19/12/2023 09:50:56  1085.7  1085.7   
2967  VN30F1M  1085.40  19/12/2023  19/12/2023 09:51:58  1085.6  1085.7   
2968  VN30F1M  1085.10  19/12/2023  19/12/2023 09:52:58  

C:\Users\Thanh\AppData\Local\Temp\ipykernel_10216\585977334.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['custom_timeframe'] = data.index // (custom_timeframe / 1)  # Tính toán số cụm 45 phút


In [62]:
print(df_grouped)

     custom_timeframe                Time    Open    High     Low   Close  \
0                 0.0 2023-12-01 09:04:58  1080.9  1082.0  1079.3  1081.8   
1                 1.0 2023-12-01 09:09:58  1082.0  1084.2  1081.8  1083.9   
2                 2.0 2023-12-01 09:14:58  1083.7  1085.6  1083.7  1085.6   
3                 3.0 2023-12-01 09:19:58  1085.6  1086.2  1084.6  1084.7   
4                 4.0 2023-12-01 09:24:55  1084.7  1084.9  1083.5  1084.2   
..                ...                 ...     ...     ...     ...     ...   
589             589.0 2023-12-19 09:33:58  1083.5  1084.4  1083.5  1084.4   
590             590.0 2023-12-19 09:38:58  1084.3  1084.8  1084.1  1084.8   
591             591.0 2023-12-19 09:43:58  1084.8  1085.3  1084.7  1085.1   
592             592.0 2023-12-19 09:48:58  1085.0  1085.8  1085.0  1085.7   
593             593.0 2023-12-19 09:53:58  1085.6  1085.9  1084.8  1084.8   

     Volume  
0      8740  
1      7425  
2      6431  
3      7338  
4    

In [63]:
df_grouped

,custom_timeframe,Time,Open,High,Low,Close,Volume
0,0.0,2023-12-01 09:04:58,1080.9,1082.0,1079.3,1081.8,8740
1,1.0,2023-12-01 09:09:58,1082.0,1084.2,1081.8,1083.9,7425
2,2.0,2023-12-01 09:14:58,1083.7,1085.6,1083.7,1085.6,6431
3,3.0,2023-12-01 09:19:58,1085.6,1086.2,1084.6,1084.7,7338
4,4.0,2023-12-01 09:24:55,1084.7,1084.9,1083.5,1084.2,5410
...,...,...,...,...,...,...,...
589,589.0,2023-12-19 09:33:58,1083.5,1084.4,1083.5,1084.4,2726
590,590.0,2023-12-19 09:38:58,1084.3,1084.8,1084.1,1084.8,2933
591,591.0,2023-12-19 09:43:58,1084.8,1085.3,1084.7,1085.1,3640
592,592.0,2023-12-19 09:48:58,1085.0,1085.8,1085.0,1085.7,3955
